In [ ]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('/content/flights_final.csv')

df.head()


,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude
0,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
1,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834
2,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599
3,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100
4,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699


Funcion

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

class Graph:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def make_set(self, vertice):
        self.parent[vertice] = vertice
        self.rank[vertice] = 0

    def find_set(self, vertice):
        if self.parent[vertice] != vertice:
            self.parent[vertice] = self.find_set(self.parent[vertice])  # Path compression
        return self.parent[vertice]

    def union(self, u, v):
        root_u = self.find_set(u)
        root_v = self.find_set(v)

        if root_u != root_v:
            if self.rank[root_u] > self.rank[root_v]:
                self.parent[root_v] = root_u
            else:
                self.parent[root_u] = root_v
                if self.rank[root_u] == self.rank[root_v]:
                    self.rank[root_v] += 1

    # Algoritmo de Kruskal
    def kruskal(self, componente, aristas_componente):
        resultante = []  # Aristas del MST
        cont = 0  # Contador de pasos

        # Inicializar cada vértice en su propio conjunto
        for vertice in componente:
            self.make_set(vertice)

        # Ordenar las aristas por peso
        ordenada = list(aristas_componente)
        ordenada.sort()  # Ordenar por el primer elemento (peso)

        # Recorrer las aristas en orden
        for dato in ordenada:
            peso, u, v = dato
            if self.find_set(u) != self.find_set(v):
                resultante.append(dato)
                cont += 1
                self.union(u, v)

        return resultante

    # Función para hacer DFS en el grafo
    def dfs(self, v, visited, grafo):
        visited.add(v)
        for u in grafo[v]:
            if u not in visited:
                self.dfs(u, visited, grafo)

    # Función para determinar si el grafo es conexo y encontrar componentes
    def es_conexo(self, grafo):
        # Crear un conjunto para marcar los vértices visitados
        visited = set()
        componentes = []

        # Recorrer todos los vértices del grafo
        for vertice in grafo:
            if vertice not in visited:
                # Realizar DFS desde un vértice no visitado
                componente_actual = set()
                self.dfs(vertice, componente_actual, grafo)
                componentes.append(componente_actual)
                visited.update(componente_actual)

        # Si solo hay una componente, el grafo es conexo
        es_conexo = len(componentes) == 1

        return es_conexo, componentes

def dijkstra(graph, start):
    # Inicializamos las distancias para todos los nodos
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0 # Distancia 0 para el nodo inicial
    prio_queue = [(0, start)]  # Cola de prioridad (distancia,nodo)




    while prio_queue:
        # Obtener el nodo que tenga la distancia mas corta
        current_distance, current_vertex = heapq.heappop(prio_queue)

        if current_distance > distances[current_vertex]:
            continue


        # Obtener los nodos adyacentes y los pesos en las aristas entre el nodo actual y el adyacente
        for adj, weight in graph[current_vertex]:
            #Calculamos la distancia que vamos a comparar
            new_distance = current_distance + weight

            # Revisamos si esta nueva distancia es mejor que la que tiene el nodo adyacente actual
            if new_distance < distances[adj]:
                #Actualizamos
                distances[adj] = new_distance

                heapq.heappush(prio_queue, (new_distance, adj))

    return distances

def create_graph(edge_list):
    graph = {}
    for weight, start, end in edge_list:
        if start not in graph:
            graph[start] = []
        if end not in graph:
            graph[end] = []
        graph[start].append((end, weight))

        graph[end].append((start, weight))
    return graph

def get_path(previous_nodes, start, end):
    path = []

    while end is not None:

        path.append(end)
        end = previous_nodes[end]
    path.reverse()
    return path


MAIN


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Leer los datos del CSV
df = pd.read_csv('/content/flights_final.csv')

# Crear un mapeo de códigos de aeropuerto a enteros (si los códigos no son numéricos)
airport_codes = pd.concat([df['Source Airport Code'], df['Destination Airport Code']]).unique()
code_to_int = {code: idx for idx, code in enumerate(airport_codes)}

# Crear el grafo usando las rutas de vuelos (grafo no dirigido)
grafo = defaultdict(list)
aristas = []  # Lista para almacenar todas las aristas con sus pesos

# Agregar las aristas (aeropuertos con la distancia entre ellos)
for index, row in df.iterrows():
    source_lat = row['Source Airport Latitude']
    source_lon = row['Source Airport Longitude']
    dest_lat = row['Destination Airport Latitude']
    dest_lon = row['Destination Airport Longitude']

    # Calcular la distancia entre los aeropuertos usando Haversine
    R = 6371  # Radio de la Tierra en km
    lat1 = np.radians(source_lat)
    lon1 = np.radians(source_lon)
    lat2 = np.radians(dest_lat)
    lon2 = np.radians(dest_lon)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c  # Distancia entre los dos puntos

    # Convertir los códigos de aeropuerto a enteros
    source_code_int = code_to_int[row['Source Airport Code']]
    dest_code_int = code_to_int[row['Destination Airport Code']]

    # Agregar la arista al grafo (no dirigido)
    grafo[source_code_int].append(dest_code_int)
    grafo[dest_code_int].append(source_code_int)

    # Agregar la arista con peso a la lista de aristas
    aristas.append((distance, source_code_int, dest_code_int))

# Inicializar la clase Graph
graph_obj = Graph()

# Determinar si el grafo es conexo y contar componentes
conexo, componentes = graph_obj.es_conexo(grafo)

# Mostrar el resultado de la conectividad
if conexo:
    print("El grafo es conexo.")
else:
    print("El grafo es NO conexo.")
    print(f"El grafo tiene un total de {len(componentes)} componentes conexas.")
    for i, componente in enumerate(componentes):
        print(f"Componente {i+1} tiene {len(componente)} vértices.")

# Ahora calculamos el MST para cada componente
if not conexo:
    for i, componente in enumerate(componentes):
        # Filtrar las aristas que pertenecen a esta componente
        aristas_componente = [(peso, u, v) for (peso, u, v) in aristas if u in componente and v in componente]

        # Aplicar Kruskal para esta componente
        mst = graph_obj.kruskal(componente, aristas_componente)

        # Calcular el peso total del MST de esta componente
        peso_total = sum(peso for peso, _, _ in mst)

        # Mostrar resultados
        print(f"Peso total del Árbol de Expansión Mínima (MST) para la Componente {i+1}: {peso_total:.2f} km")


El grafo es NO conexo.
El grafo tiene un total de 7 componentes conexas.
Componente 1 tiene 3230 vértices.
Componente 2 tiene 4 vértices.
Componente 3 tiene 4 vértices.
Componente 4 tiene 10 vértices.
Componente 5 tiene 4 vértices.
Componente 6 tiene 2 vértices.
Componente 7 tiene 2 vértices.
Peso total del Árbol de Expansión Mínima (MST) para la Componente 1: 1197584.25 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 2: 1598.90 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 3: 787.25 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 4: 1441.06 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 5: 255.94 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 6: 98.79 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 7: 37.83 km


In [ ]:
airport_info = {}
for index, row in df.iterrows():
    code = str(row['Source Airport Code'])
    airport_info[code] = {
        'name': str(row['Source Airport Name']),
        'city': str(row['Source Airport City']),
        'country': str(row['Source Airport Country']),
        'lat': str(row['Source Airport Latitude']),
        'lon': str(row['Source Airport Longitude']),
    }
codigo=input("Escriba el codigo de algun aeropuerto")
graph2=create_graph(aristas)
for index, row in df.iterrows():
  if codigo==str(row['Source Airport Code']):
    name= str(row['Source Airport Name'])
    city= str(row['Source Airport City'])
    country= str(row['Source Airport Country'])
    lat = str(row['Source Airport Latitude'])
    lon = str(row['Source Airport Longitude'])
    c=code_to_int[codigo]
    dist=dijkstra(graph2,c)
    break
print("Informacion del punto inicial:")
print("Codigo:"+codigo+", Nombre:"+name+", Ciudad"+city+", Pais:"+country+", Latitud:"+lat+", Longitud:"+lon)

filtered_data = {k: v for k, v in dist.items() if v != float('inf')}
longest_paths = heapq.nlargest(10, filtered_data.items(), key=lambda item: item[1])
int_to_code = {idx: code for code, idx in code_to_int.items()}

print(longest_paths)
print("Informacion de destinos mas lejanos de los caminos minimos:")

for cde, distancia in longest_paths:
  converted= int_to_code.get(cde, "Code not found")

  if converted in airport_info:
      info = airport_info[converted]
      name= info['name']
      city= info['city']
      country= info['country']
      lat = info['lat']
      lon = info['lon']
      print("Codigo:"+converted+", Nombre:"+name+", Ciudad"+city+", Pais:"+country+", Latitud:"+lat+", Longitud:"+lon+", Distancia:"+str(distancia))



Escriba el codigo de algun aeropuertoCOK
Informacion del punto inicial:
Codigo:COK, Nombre:Cochin International Airport, CiudadKochi, Pais:India, Latitud:10.152, Longitud:76.401901
[(3024, 19627.054781390147), (2894, 19238.09703060891), (2933, 18965.660957928983), (1686, 18807.594248652284), (2346, 18781.417660857813), (1473, 18717.668371371466), (1661, 18676.96706131807), (1035, 18661.35388150012), (2755, 18611.30133002254), (2583, 18603.698694022143)]
Informacion de destinos mas lejanos de los caminos minimos:
Codigo:IPC, Nombre:Mataveri Airport, CiudadEaster Island, Pais:Chile, Latitud:-27.16480064, Longitud:-109.4219971, Distancia:19627.054781390147
Codigo:MPN, Nombre:Mount Pleasant Airport, CiudadMount Pleasant, Pais:Falkland Islands, Latitud:-51.82279968, Longitud:-58.44720078, Distancia:19238.09703060891
Codigo:TBP, Nombre:Capitan FAP Pedro Canga Rodriguez Airport, CiudadTumbes, Pais:Peru, Latitud:-3.55253005, Longitud:-80.38140106, Distancia:18965.660957928983
Codigo:PIU, Nombr

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

# Función para mostrar el mapa con aeropuertos y rutas más largas
def plot_map_with_routes(airport_info, longest_paths):
    # Mapbox access token (reemplaza con el tuyo si es necesario)
    mapbox_access_token = 'pk.eyJ1Ijoic2ViYXN0aWFubWFsZG9uYWRvMTk0NSIsImEiOiJjbGluYnRobHkwbDQyM2xwOGc4aGN5ZnpvIn0.Jal1X7da0VhVK8gkKrWBng'

    # Crear figura de Plotly
    fig = go.Figure()

    # Añadir marcadores para cada aeropuerto
    for code, info in airport_info.items():
        fig.add_trace(go.Scattermapbox(
            lon=[info['lon']],
            lat=[info['lat']],
            mode='markers+text',
            marker=dict(size=10, color='red', symbol='airport'),
            text=[info['name']],
            textposition='bottom right'
        ))

    # Dibujar las rutas más largas en el mapa
    for code, dist in longest_paths:
        info_start = airport_info[codigo]  # Info del aeropuerto de origen
        info_end = airport_info[int_to_code[code]]  # Info del aeropuerto de destino

        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=[info_start['lon'], info_end['lon']],
            lat=[info_start['lat'], info_end['lat']],
            line=dict(width=2, color='blue'),
            hoverinfo='none'
        ))

    # Configuración del mapa
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style="outdoors",
            zoom=1,
            center=dict(lat=0, lon=0)
        ),
        showlegend=False
    )

    # Mostrar el mapa
    fig.show()

# Continuación de tu código anterior...

# Creamos el diccionario de información del aeropuerto
airport_info = {}
for index, row in df.iterrows():
    code = str(row['Source Airport Code'])
    airport_info[code] = {
        'name': str(row['Source Airport Name']),
        'city': str(row['Source Airport City']),
        'country': str(row['Source Airport Country']),
        'lat': str(row['Source Airport Latitude']),
        'lon': str(row['Source Airport Longitude']),
    }

codigo = input("Escriba el código de algún aeropuerto: ")
graph2 = create_graph(aristas)

# Obtener la información del aeropuerto seleccionado
for index, row in df.iterrows():
    if codigo == str(row['Source Airport Code']):
        name = str(row['Source Airport Name'])
        city = str(row['Source Airport City'])
        country = str(row['Source Airport Country'])
        lat = str(row['Source Airport Latitude'])
        lon = str(row['Source Airport Longitude'])
        c = code_to_int[codigo]
        dist = dijkstra(graph2, c)
        break

# Mostrar la información del aeropuerto inicial
print("Información del punto inicial:")
print(f"Código: {codigo}, Nombre: {name}, Ciudad: {city}, País: {country}, Latitud: {lat}, Longitud: {lon}")

# Filtrar los destinos más lejanos de los caminos mínimos
filtered_data = {k: v for k, v in dist.items() if v != float('inf')}
longest_paths = heapq.nlargest(10, filtered_data.items(), key=lambda item: item[1])
int_to_code = {idx: code for code, idx in code_to_int.items()}

print("Información de destinos más lejanos de los caminos mínimos:")
for cde, distancia in longest_paths:
    converted = int_to_code.get(cde, "Code not found")

    if converted in airport_info:
        info = airport_info[converted]
        name = info['name']
        city = info['city']
        country = info['country']
        lat = info['lat']
        lon = info['lon']
        print(f"Código: {converted}, Nombre: {name}, Ciudad: {city}, País: {country}, Latitud: {lat}, Longitud: {lon}, Distancia: {distancia:.2f} km")

# Llamar a la función para graficar el mapa con los nodos y las rutas más largas
plot_map_with_routes(airport_info, longest_paths)


Escriba el código de algún aeropuerto: SDQ


NameError: name 'create_graph' is not defined